<a href="https://colab.research.google.com/github/tungreng/personal-porfolio/blob/main/Shopee_Seller_LTV_(Ads)_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Creator: Nga Vu <br>
Check out my course at: https://200lab.io/khoa-hoc-data-analyst-co-ban-sql-google-data-studio <br>
Challenge trên Kaggle - <b>Seller LTV (Ads)</b>: https://www.kaggle.com/competitions/shopee-case-study-seller-ltv-ads <br>
Download data : https://www.kaggle.com/c/shopee-case-study-seller-ltv-ads-jan21
Đầu tiên bạn phải tham gia vào Challenge này mới có quyền Download dataset nhé  
## Background
Quảng cáo trả phí (Paid Advertising) là một nguồn thu chính của Shopee. Do đó, điều quan trọng là phải xác định các nhóm Seller quan trọng, cho phép chúng tôi cải thiện mức độ tương tác của người bán trong việc phát triển, giữ chân và thu hút được những Seller này.
# Model Output  
Vì chúng tôi muốn biết chi tiêu quảng cáo hàng tháng của mỗi người bán trong 3 tháng tới, mô hình nên bao gồm ba nhãn Y được dự đoán, một nhãn cho mỗi tháng tiếp theo (FXM). Ví dụ: nếu ngày hiện tại là ngày 1 tháng 8 năm 2020, các nhãn Y được dự đoán như sau:

Nhãn Y F1M: Chi tiêu quảng cáo dự đoán cho tháng 8 năm 2020
Nhãn Y F2M: Dự đoán chi tiêu cho quảng cáo trong tháng 9 năm 2020
Nhãn Y F3M: Chi tiêu quảng cáo dự đoán cho tháng 10 năm 2020
## Data Definition
### Settings
Tập dữ liệu train và test trong nghiên cứu này được đặt tên lần lượt là xx và yy. Những dữ liệu này được trích xuất dựa trên các setting được mô tả trong biểu đồ này.
<img width="1000" src="https://www.googleapis.com/download/storage/v1/b/kaggle-user-content/o/inbox%2F5897968%2F699d504caa33db1cd099358881284d49%2Fregression%20settings.jpg?generation=1602577603351979&alt=media">

## Features
Có 525 cột trong tập dữ liệu:

- `shop_id`: Đây là số định danh duy nhất cho mỗi người bán
- `shopprofilex`: Có 27 features liên quan đến hồ sơ cửa hàng. Ví dụ các feature về loại cửa hàng (tức là xuyên biên giới, chính thức, ưu tiên), danh mục mặt hàng chính, xếp hạng, người theo dõi, ngày đăng ký.
- `shopbehaviorx`: Có 51 feature liên quan đến hành vi của cửa hàng. Ví dụ các feature về loại hình khuyến mãi được sử dụng (giảm giá trên sản phẩm, voucher), số lần đăng nhập, số lượng sản phẩm. Các feature này được trích xuất trong các khoảng thời gian khác nhau (Ví dụ 1 tuần trước, 1 tháng trước).
- `shopperformancex`: Có 73 feature liên quan đến hiệu suất của cửa hàng. Ví dụ số lượng đơn đặt hàng, tổng giá trị hàng hóa (GMV), số lần hiển thị sản phẩm, số lần nhấp vào sản phẩm và lượt xem sản phẩm. Các feature này được trích xuất trong các khoảng thời gian khác nhau.
- `organicperformancex`: Có 24 feature liên quan đến hiệu suất thuần (organic). Organic performance- còn được gọi là hiệu suất không được quảng cáo (non-advertised performance). Ví dụ về các feature này tương tự như các feature trong shopperformancex.
- `adsperformancex`: Có 171 tính năng liên quan đến hiệu suất quảng cáo. Hiệu suất quảng cáo là hiệu suất của cửa hàng - kết quả của quảng cáo có trả tiền. Ví dụ về các feature này tương tự như các feature của shopperformancex.
- `adsbehaviorx`: Có 60 feature liên quan đến hành vi quảng cáo. Ví dụ như số lượng quảng cáo đang hoạt động, số lần chỉnh sửa (tức là điều chỉnh từ khóa, giá thầu (bid price)) và ngày tạo tài khoản quảng cáo. Một số features này được trích xuất trong các khoảng thời gian khác nhau.
- `adstopupx`: Có 115 feature liên quan đến topup quảng cáo. Ví dụ như phương thức nạp tiền, số tiền nạp tiền, ngày nạp tiền lần cuối và số dư tín dụng còn lại.
- `adsexpensefuture_1m`: Giá trị Y thực tế cho tháng chuyển tiếp đầu tiên
- `adsexpensefuture_2m`: Giá trị Y thực tế cho tháng thứ hai trở đi
- `adsexpensefuture_3m`: Giá trị Y thực tế cho tháng thứ ba trở đi


In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
# Import thư viện
import numpy as np
import pandas as pd
from tqdm import tqdm
pd.set_option('display.max_colwidth', None)
from sklearn.model_selection import train_test_split

In [ ]:
# Upload data to your Google Drive and change the path
train = pd.read_csv('/content/drive/MyDrive/Data/shopee-case-study-seller-ltv/201014_train_set.csv')
print(f"Train shape: {train.shape}")
train.head(10)

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (6,415) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Train shape: (180767, 525)


,shop_id,shop_profile_1,shop_profile_2,shop_profile_3,shop_profile_4,shop_profile_5,shop_profile_6,shop_profile_7,shop_profile_8,shop_profile_9,...,ads_topup_109,ads_topup_110,ads_topup_111,ads_topup_112,ads_topup_113,ads_topup_114,ads_topup_115,ads_expense_1m_future,ads_expense_2m_future,ads_expense_3m_future
0,0,1717,342.0,544177.10,0,0.997959,1.0,Y,2.0,0,...,-4.788862,3.0,35.436387,0.000000,71,2426.279037,43.099759,575.850954,838.687882,524.638741
1,1,1717,43.0,129335.57,0,0.917606,1.0,Y,2.0,0,...,0.000000,10.0,0.000000,0.000000,9,70.629380,14.366586,0.000000,0.000000,42.289979
2,2,1717,6.0,105615.56,0,0.508388,1.0,Y,2.0,0,...,0.000000,0.0,0.000000,0.000000,2,21.411167,0.000000,0.000000,0.000000,5.781112
3,3,1717,0.0,366974.78,0,0.383161,0.0,N,2.0,0,...,0.000000,5.0,0.000000,0.000000,1,6.947319,0.000000,0.000000,0.000000,0.000000
4,4,1717,16.0,328581.68,0,0.497274,0.0,Y,2.0,0,...,0.000000,1.0,3.543639,0.000000,5,17.690951,0.000000,1.811587,0.000000,0.000000
5,5,1717,0.0,326417.37,0,0.122905,1.0,N,2.0,0,...,0.000000,0.0,0.000000,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000
6,6,1717,45.0,175213.11,0,0.792880,1.0,Y,2.0,0,...,0.000000,12.0,0.000000,0.000000,2,10.705584,7.183293,7.785920,1.168767,0.000000
7,7,1717,0.0,150039.63,0,0.413713,0.0,N,2.0,0,...,0.000000,1.0,3.543639,0.000000,2,10.572645,0.000000,0.000000,0.000000,0.000000
8,8,1717,202.0,112027.75,0,0.999054,1.0,Y,2.0,0,...,-2.394431,2.0,248.054711,851.428818,198,6974.535839,280.148431,759.155228,497.910951,512.434386
9,9,1717,0.0,135697.80,0,0.638074,0.0,Y,2.0,0,...,0.000000,16.0,0.000000,0.000000,8,100.924308,0.000000,0.000000,0.000000,0.000000


## Feature Engineering

In [ ]:
# remove columns with only one unique  value and shop_id
drops = []
for col in list(train.columns):
  if len(train[col].value_counts())==1:
    drops.append(col)
train.drop(drops+['shop_id'],axis=1,inplace=True)

In [ ]:
# Count columns by data type
train.dtypes.value_counts()

float64    449
int64       45
object       6
dtype: int64

In [ ]:
numeric_types = ['float64','int64']
numeric_cols = train.select_dtypes(numeric_types).columns.tolist()
print(len(numeric_cols))

494


In [ ]:
categorical_types = ['object','bool']
categorical_cols = train.select_dtypes(categorical_types).columns.tolist()
print(len(categorical_cols))
categorical_cols

6


['shop_profile_6',
 'shop_profile_7',
 'shop_profile_8',
 'shop_profile_16',
 'shop_profile_27',
 'ads_topup_9']

### Convert to Right Data Type

In [ ]:
# Check null columns
train.columns[train.isnull().sum()>=1]

Index(['shop_profile_13', 'shop_profile_23', 'shop_profile_25',
       'ads_behavior_39', 'ads_topup_3', 'ads_topup_8', 'ads_topup_21',
       'ads_topup_22', 'ads_topup_26', 'ads_topup_36', 'ads_topup_45',
       'ads_topup_52', 'ads_topup_64', 'ads_topup_72', 'ads_topup_85',
       'ads_topup_89', 'ads_topup_90', 'ads_topup_94', 'ads_topup_96',
       'ads_topup_104', 'ads_topup_110'],
      dtype='object')

In [ ]:
# deal with float
for col in numeric_cols:
  if train[col].dtype == 'float64':
    train[col] = train[col].astype(np.float64,errors='raise')


In [ ]:
# deal with int
for col in numeric_cols:
  if train[col].dtype == 'int64':
    train[col] = train[col].astype(np.int64,errors='raise')

In [ ]:
train['ads_topup_9']=train['ads_topup_9'].apply(lambda x: str(x).strip())=='1.0'
train['shop_profile_6']=train['shop_profile_6'].apply(lambda x: str(x).strip())=='1.0'
train['shop_profile_7'] = train['shop_profile_7']=='Y'
train['shop_profile_16'] = train['shop_profile_16']=='True'
train['shop_profile_27'] = train['shop_profile_27']=='True'
train['shop_profile_8'] = train['shop_profile_8'].replace({'unknown':np.nan})

In [ ]:
# Check null percentage
train[categorical_cols].isnull().mean().sort_values(ascending=False)

shop_profile_8     0.097734
shop_profile_6     0.000000
shop_profile_7     0.000000
shop_profile_16    0.000000
shop_profile_27    0.000000
ads_topup_9        0.000000
dtype: float64

### Feature Selection

In [ ]:
# Threshold for removing correlated variables
threshold = 0.9

# Absolute value correlation matrix
corr_matrix = train.drop(['ads_expense_1m_future','ads_expense_2m_future','ads_expense_3m_future'],axis=1).corr().abs()
corr_matrix.head()

,shop_profile_1,shop_profile_2,shop_profile_3,shop_profile_4,shop_profile_5,shop_profile_6,shop_profile_7,shop_profile_9,shop_profile_10,shop_profile_11,...,ads_topup_106,ads_topup_107,ads_topup_108,ads_topup_109,ads_topup_110,ads_topup_111,ads_topup_112,ads_topup_113,ads_topup_114,ads_topup_115
shop_profile_1,1.000000,0.029435,0.012319,0.004704,0.055306,0.013985,0.057959,0.143909,0.079565,0.004774,...,0.289438,0.016481,0.006610,0.000621,0.101768,0.037315,0.002584,0.029492,0.007550,0.003658
shop_profile_2,0.029435,1.000000,0.002699,0.004811,0.239302,0.126504,0.208910,0.155860,0.423030,0.069698,...,0.194194,0.061515,0.087706,0.022435,0.030932,0.061205,0.064904,0.135445,0.092321,0.076332
shop_profile_3,0.012319,0.002699,1.000000,0.001481,0.003081,0.007246,0.000376,0.014389,0.014387,0.672912,...,0.002647,0.009952,0.003785,0.000286,0.005275,0.003646,0.001154,0.012031,0.003350,0.002128
shop_profile_4,0.004704,0.004811,0.001481,1.000000,0.011240,0.020755,0.012281,0.045646,0.002213,0.006169,...,0.001146,0.002747,0.016980,0.014609,0.053369,0.050388,0.007723,0.005176,0.025730,0.015978
shop_profile_5,0.055306,0.239302,0.003081,0.011240,1.000000,0.232187,0.880093,0.000430,0.488541,0.143460,...,0.569051,0.222105,0.222365,0.051381,0.087064,0.130715,0.167025,0.467306,0.195551,0.198188


In [ ]:
# Upper triangle of correlations
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
to_drop = [column for column in upper.columns if any(upper[column] > threshold)]
print('There are %d columns to remove.' % (len(to_drop)))
train = train.drop(columns = to_drop,axis=1)

<ipython-input-21-cabde9f66f3a>:2: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))


There are 224 columns to remove.


In [ ]:
len(train.columns)

276

#Train Model

In [ ]:
from sklearn.model_selection import train_test_split
X, y = train.drop(['ads_expense_1m_future','ads_expense_2m_future','ads_expense_3m_future'],axis=1), train['ads_expense_1m_future']
x_train, x_test, y_train, y_test = train_test_split(X,y, random_state=0,test_size=.2)

In [ ]:
import numpy as np
np.random.seed(0)
import random
random.seed(0)
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, mean_squared_error
from sklearn.pipeline import Pipeline
import time

In [ ]:
# Remove predicted columns from numeric columns
numeric_cols=numeric_cols[:-3]

In [ ]:
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import explained_variance_score,mean_absolute_error,r2_score
from time import time
# deal with binary columns
binary_pipeline = Pipeline([
    ('select_categorical_features',FunctionTransformer(lambda df:df[categorical_cols])),
    ('fillna',SimpleImputer(strategy='most_frequent'))
])
# deal with numerical columns
numerical_pipeline = Pipeline([
    ('select_numerical_features',FunctionTransformer(lambda df:df[numeric_cols])),
    ('impute',SimpleImputer(strategy='constant',fill_value=999)),
     ('scale', StandardScaler()),
])

simple_fe = FeatureUnion([
    ('binary_pipeline',binary_pipeline),
    ('numerical_pipeline',numerical_pipeline)
])
regressors = [
        KNeighborsRegressor(),
        GradientBoostingRegressor(),
        KNeighborsRegressor(),
        ExtraTreesRegressor(),
        RandomForestRegressor(),
        DecisionTreeRegressor(),
        LinearRegression(),
        Lasso(),
        Ridge()]

for model in regressors:
    start = time()
    parsed_x_train = simple_fe.fit_transform(x_train,y_train)
    parsed_x_test = simple_fe.fit_transform(x_test,y_test)
    model.fit(parsed_x_train, y_train)
    train_time = time() - start
    start = time()
    y_pred = model.predict(parsed_x_test)
    predict_time = time()-start
    print(model)
    print("\tTraining time: %0.3fs" % train_time)
    print("\tPrediction time: %0.3fs" % predict_time)
    print("\tExplained variance:", explained_variance_score(y_test, y_pred))
    print("\tMean absolute error:", mean_absolute_error(y_test, y_pred))
    print("\tR2 score:", r2_score(y_test, y_pred))
    print()

KNeighborsRegressor()
	Training time: 6.104s
	Prediction time: 191.443s
	Explained variance: 0.658112120530549
	Mean absolute error: 10.73100932219858
	R2 score: 0.6571890418285817

GradientBoostingRegressor()
	Training time: 413.403s
	Prediction time: 0.823s
	Explained variance: 0.8091926684409598
	Mean absolute error: 8.38592879762168
	R2 score: 0.8091534132377277

KNeighborsRegressor()
	Training time: 7.336s
	Prediction time: 190.108s
	Explained variance: 0.658112120530549
	Mean absolute error: 10.73100932219858
	R2 score: 0.6571890418285817

ExtraTreesRegressor()
	Training time: 897.689s
	Prediction time: 1.913s
	Explained variance: 0.8091138548339784
	Mean absolute error: 8.989786505203513
	R2 score: 0.8091104341286488

RandomForestRegressor()
	Training time: 2171.275s
	Prediction time: 2.018s
	Explained variance: 0.8093290459327699
	Mean absolute error: 13.060350018610809
	R2 score: 0.8086455047845943

DecisionTreeRegressor()
	Training time: 45.212s
	Prediction time: 0.340s
	Expl